In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

Numpy jest bardzo wydajny przy odpowiednim użyciu. Dzisiaj postaramy się zgłębić w jaki sposób działa aby korzystać z niego w efektywny sposób nawet przy skomplikowanych problemach

In [ ]:
%%timeit
l = [0 for _ in range(10000)]
for i in range(10000):
    l[i] += 5

In [ ]:
%%timeit
l = np.zeros(10000)
l += 5

# Wstęp

Tablica w numpy to nie tylko dane. Poza blokiem w pamięci w którym przechowywane są dane mamy także metainformacje służące do wydajnego korzystania z tablicy. W szczególności jak znaleźć i interpretować element

In [ ]:
x = np.array([0,1,2,3,4.99])
print(x.__array_interface__)
print(x.data)
print(x.strides)

In [ ]:
x = np.array([0,1,2,3,4], np.int8)
print(x.__array_interface__)
print(x.data)
print(x.strides)

#### Zad 1
Zaimplementuj indeksację macierzy. \
Stwórzmy prymitywną strukturę danych przypominającą macierz numpy. Składa się ona z danych oraz wymiarów macierzy. Teraz Twoim zadaniem jest umożliwienie odwołania się do konkretnego elementu macierzy podając wiersz oraz kolumnę

In [ ]:
from itertools import product
l = list(map(lambda x: x[0] + str(x[1]), product('ABCD', np.arange(7))))
x = {
    'data': l,
    'nrows': 4,
    'ncols': 7
}
x

In [ ]:
def at(x, row, col):
    return 0 # Your code goes here

In [ ]:
assert at(x, 0, 0) == 'A0'
assert at(x, 0, 1) == 'A1'
assert at(x, 2, 2) == 'C2'
assert at(x, 2, 3) == 'C3'
assert at(x, 3, 6) == 'D6'
assert at(x, 3, 5) == 'D5'

#### Zad 2
Świetnie, teraz spróbujemy dokonać manipulacji bez ingerencji w dane. Transpozycja to w skrócie zamienienie miejscami wierszy z kolumnami. Czy jesteś w stanie stworzyć funkcję, która bez ingerencji w danych zwróci komórkę z podanego wiersza i kolumny ale z transponowanej macierzy?

In [ ]:
npx = np.array(x['data']).reshape(x['nrows'], x['ncols'])
npx

In [ ]:
npx.T

In [ ]:
npx.T[1,3]

In [ ]:
npx.T[2,0]

In [ ]:
def atTransposed(x, row, col):
    pass # Miejsce na implementację

In [ ]:
assert atTransposed(x, 1, 3) == 'D1'
assert atTransposed(x, 2, 0) == 'A2'

Udało nam się transponować macierz bez ingerencji w dane. Operacja ta będzie działać bardzo szybko niezależnie od rozmiarów macierzy. Poniżej pokazuję ciekawostkę aby pokazać, że numpy rzeczywiście działa niskopoziomowo w zbliżony sposób do tego przedstawionego powyżej

In [ ]:
x = np.array([0,1,2,3], dtype=np.int8)
x

In [ ]:
x.dtype = "<i2"

In [ ]:
x

In [ ]:
0x0100, 0x0302

Nakazaliśmy inną interpretację danych i rzeczywiście się to stało bez żadnej ingerencji w bitowy zapis samych danych

## Widoki
Dzięki takiemu podejściu możliwe jest dzielenie danych przez kilka tablic, co pozwala na oszczędność pamięci

In [ ]:
x = np.array([0,1,2,3], dtype=np.int8)
x

In [ ]:
y = x.view()
y

In [ ]:
x[0] = 2
y

In [ ]:
y.dtype = "<i2"
y

In [ ]:
x

In [ ]:
x[1] = 3
x

In [ ]:
y

In [ ]:
y[1] = 356
y

In [ ]:
x

## Strides

In [ ]:
x = np.arange(9, dtype=np.int8)
x

In [ ]:
x.tobytes()

In [ ]:
x.strides

In [ ]:
x = x.reshape(3,3)
x

In [ ]:
x.tobytes()

In [ ]:
x.strides

Atrybut strides w połączeniu z wymiarami tablicy pozwala się po niej poruszać i odwołać do konkretnego elementu. Tutaj mamy już wykorzystanie niskopoziomowej reprezentacji - dowiadujemy się o ile bajtów musimy się przesunąć, aby przejść do kolejnego elementu w danym wymiarze

In [ ]:
x = np.arange(9, dtype=np.int16).reshape(3,3)
x

In [ ]:
x.tobytes()

In [ ]:
x.strides

In [ ]:
x = np.arange(9, dtype=np.int16)
x.strides

In [ ]:
y = x[::-1]
y

In [ ]:
y.strides

Jak widać nadal różnego rodzaju modyfikacje odbywają się nie poprzez ingerencję w dane a w metainformacje

In [ ]:
from numpy.lib.stride_tricks import as_strided
help(as_strided)

Za pomocą funkcji as_strided możemy utworzyć nowy widok na tablicę o określonych przez nas atrybutach strides oraz wymiarach

In [ ]:
x = np.arange(9, dtype=np.int16)
as_strided(x, shape=(3,), strides=(4,))

In [ ]:
as_strided(x[1:], (3,), (6,))

In [ ]:
x = np.arange(36, dtype=np.int32).reshape(6, 6)
x

#### Zad 3
Korzystając z funkcji as_strided utwórz spodziewane widoki

In [ ]:
expected = np.array([0,1,2,3])
expected

In [ ]:
y = as_strided(x, (1,), (1,)) # Do implementacji
assert (y == expected).all()

In [ ]:
expected = np.array([0,7,14,21,28,35])
expected

In [ ]:
y = as_strided(x, (1,), (1,)) # Do implementacji
assert (y == expected).all()

In [ ]:
expected = np.array([
    [0,2],
    [7,9],
    [14,16]])
expected

In [ ]:
y = as_strided(x, (1,), (1,)) # Do implementacji
assert (y == expected).all()

In [ ]:
expected = np.array([
    [0,0,0],
    [12,12,12],
    [24,24,24]])
expected

In [ ]:
y = as_strided(x, (1,), (1,)) # Do implementacji
assert (y == expected).all()

In [ ]:
expected = np.array([np.arange(12).reshape(2,6)+x*6 for x in range(3)])
expected

In [ ]:
y = as_strided(x, (1,), (1,)) # Do implementacji
assert (y == expected).all()

### Ciekawostka

In [ ]:
x = np.random.randn(10000)
y = np.random.randn(10000*33)[::33]

In [ ]:
x.shape, y.shape

In [ ]:
x.strides, y.strides

In [ ]:
%timeit x.sum()

In [ ]:
%timeit y.sum()

Dlaczego sumowanie y zajmuje dłużej? Podpowiedź: cache procesora

Nieco wygodniejszym w zastosowaniu przez działanie na wyższym poziomie abstrakcji jest sliding_window_view

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

In [ ]:
x = np.arange(10000)

In [ ]:
y = sliding_window_view(x, 50)
y

In [ ]:
y.shape, y.strides

Za jej pomocą można policzyć np średnią kroczącą w bardzo wydajny sposób

In [ ]:
%timeit y.mean(1)

Ciekawostka: akurat ten efekt można osiągnąć wykorzystując funkcję splotu

In [ ]:
%timeit np.convolve(x, np.ones(50)/50, 'valid')

#### Zad4
Zabierzmy się zatem za coś trudniejszego, czego nie idzie tak łatwo osiągnąć konwolucją, a do czego idealnie nadaje się sliding_window_view. \
Wyobraźmy sobie, że monitorujesz rurociąg transportujący ropę. Dane ciśnienia z punktu pomiarowego masz w zmiennej $x$. Zmienna $y$ z kolei zawiera wzorzec zdarzenia wyciekowego. Aby określić czy zaszedł wyciek należy policzyć korelację ze zdefiniowanym wzorcem. Znajdź 4 rozłączne fragmenty szeregu $x$ o długości 50 z najwyższą korelacją z zadanym wzorcem $y$ - czyli fragmenty podejrzane o bycie wyciekiem

In [ ]:
x = np.sin(np.linspace(17, 29, 100000)) + np.sin(np.linspace(0, 1000, 100000)) + np.sin(np.linspace(13, 1500, 100000)) + np.cos(np.linspace(2,173, 100000))
plt.plot(x)

In [ ]:
y = np.linspace(-1.5,1.5,50)**2

Pomocniczo zaimplementuj funkcję, która mając daną 2 wymiarową macierz oraz wzorzec liczy korelację pearsona wzorca z każdym wierszem macierzy

## Obliczenia

In [ ]:
np.random.seed(44)
df = pd.DataFrame({'pActive': np.random.rand(13), 'popularity': np.random.rand(13)})
df

Rozważmy uproszczoną modelową sytuację wyboru np prezydenta miasta. W naszym przypadku mamy 12 potencjalnych kandydatów. Każdy z nich ma przypisane prawdopodobieństwo $pActive$ określające szanse na wystartowanie w wyborach - składa się na to np szansa na zebranie podpisów, bycie wystawionym przez swój komitet itp. Poza tym określiliśmy też popularność każdego kandydata. Prawdopodobieństwo wygrania wyborów jest równe stosunkowi jego popularności do sumy popularności wszystkich aktywnych kandydatów biorących udział w wyborach.
#### Zad 5
Dla każdego kandydata policz prawdopodobieństwo wygranej. Postaraj się skorzystać jak najbardziej z wydajności numpy i przyspiesz obliczenia

Przykładowo dla dwóch potencjalnych kandydatów o pActive 0.7 i 0.2 oraz popularity 0.1 i 0.4 mamy 4 możliwe sytuacje:
 - nikt nie wystartuje w wyborach prawdopodobieństwo (1-0.7) * (1-0.2) = 0.24
 - wystartuje tylko kandydat 1 prawdopodobieństwo 0.7 * (1-0.2) = 0.56
 - wystartuje tylko kandydat 2 prawdopodobieństwo (1-0.7) * 0.2 = 0.06
 - wystartują obaj 0.7 * 0.2 = 0.14

W pierwszej sytuacji nie wygra nikt, w drugiej i trzeciej na 100% wygra jedyny kandydat, w ostatniej na 20% wygra kandydat pierwszy i na 80% kandydat drugi. Ostateczne prawdopodobieństwo wygranej to:
- dla kandydata 1: 0.56 * 100% + 0.14 * 20% = 58.8%
- dla kandydata 2: 0.06 * 100% + 0.14 * 80% = 17.2%

In [ ]:
def getP(df):
    return 0

In [ ]:
getP(df)

## Pandas

Na zajęciach będziemy wykorzystywać okrojony i zmodyfikowany zbiór danych Million Song Dataset (MSD)
 * unique_tracks.txt – zawiera informacje takie jak identyfikator utworu, identyfikator wykonania, nazwę artysty oraz tytuł utworu, https://www.cs.put.poznan.pl/kdembczynski/lectures/data/unique_tracks.zip
 * triplets_sample_20p.txt – zawiera identyfikator użytkownika, identyfikator utworu oraz datę odsłuchania. https://www.cs.put.poznan.pl/kdembczynski/lectures/data/triplets_sample_20p.zip

In [ ]:
import pandas as pd

#### Zad
Podaj 10 najpopularniejszych utworów wraz z artystą i liczbą odsłuchań

#### Zad
Podaj 10 użytkowników z największą liczbą odtworzonych unikatowych utworów

#### Zad
Który artysta ma najwięcej odtworzeń i ile?

#### Zad
Podaj sumaryczną liczbę odsłuchań w poszczególnych miesiącach

#### Zad
Podaj wszystkich użytkowników, którzy odsłuchali wszystki trzy najpopularniejsze utwory zespołu Queen

https://www.cs.put.poznan.pl/kdembczynski/lectures/mmds/
https://scipy-lectures.org/advanced/index.html